In [ ]:
get_matching_task_profile_info()
displaySaveResults()

In [ ]:
from datautils import*
import os
import os.path as path
from learningutils import *
from sklearn import tree
import json
from matching_task import *
import time

matching_tasks_summary = pd.DataFrame(columns=['Dataset', 'Domain', '#records_source', '#records_target', 'count_record_pairs', '#match', '#non-match',
                                               'count_attr','#short_string_attr', '#long_string_attr', '#numeric_attr','avg_density_all'])
matching_tasks_baseline_rf_results = pd.DataFrame(columns=['Dataset', 'precision','recall','f1','f1_std','proba_scores',
                                                          'proba_scores_std','x-val f1','x-val f1 sigma'])
matching_tasks_baseline_svm_results = pd.DataFrame(columns=['Dataset', 'precision','recall','f1','f1_std','proba_scores',
                                                      'proba_scores_std','x-val f1','x-val f1 sigma'])

matching_tasks_profiling = pd.DataFrame(columns=['Dataset','F1_xval_max', 'F1_xval_top_matching_relevant_features', 
                                                 'matching_relevant_features', 
   'matching_relevant_attributes','matching_relevant_attributes_density','matching_relevant_attributes_count',
 'matching_relevant_attributes_datatypes','top_matching_relevant_features','top_relevant_attributes', 
 'top_relevant_attributes_count','top_relevant_attributes_datatypes', 'top_relevant_attributes_density',
'avg_length_tokens_top_relevant_attributes','avg_length_words_top_relevant_attributes','corner_cases_top_matching_relevant_features'])
                                                           
# Use the flags below to indicate which results should be calculates
summaryFeatures=True
baselineResults=False
profilingFeatures = True                                                           


main_path = '/work/aprimpel/datasets_for_notebooks/datasetProfilingtrain_test_val/'
dataset_dict_path='/work/aprimpel/datasets_for_notebooks/datasetProfilingtrain_test_val/leipzig_datasets/dataset_dict.txt' # contains metadata for the distinct tasks
source_folder="leipzig_datasets"

#add the correct separators of the source sets and the gold standard
sep_for_source_files= ','
gs_sep = ','
train_test_val=True # otherwise nested x-validation for baseline experiments

#change for allowing multithreading
threads=-1

with open(dataset_dict_path) as json_file:
    dict_ = json.load(json_file)

def get_matching_task_profile_info():
     
    dat_counter = 0
    for r, d, f in os.walk(main_path+source_folder+'/'):
        print("Current path :"+r)
            
        if r!=main_path+source_folder+'/':
            dataset_name = r.split("/")[-1]
            dataset_ext_name =  dict_.get(dataset_name).get("ext_name")
            dataset_domain = dict_.get(dataset_name).get("domain")
            dataset_primAttr = dict_.get(dataset_name).get("primattr")

        else: continue
        ds1=pd.DataFrame()
        ds2=pd.DataFrame()
        gs=pd.DataFrame()
        for file in f:

            feature_vector_train,feature_vector_test,feature_vector_validation = None, None,  None

            if '.csv' in file and 'standard_2' in file:
                gs = pd.read_csv(r+'/'+file, sep=gs_sep)

            if path.exists(r+"/feature_vector_train.csv"):
                feature_vector_train = pd.read_csv(r+"/feature_vector_train.csv", sep =",", engine='python')
            if path.exists(r+"/feature_vector_test.csv"):
                feature_vector_test = pd.read_csv(r+"/feature_vector_test.csv", sep =",", engine='python')
            if path.exists(r+"/feature_vector_validation.csv"):
                feature_vector_validation = pd.read_csv(r+"/feature_vector_validation.csv", sep =",", engine='python')
            if '.csv' in file and 'standard' not in file and 'feature' not in file:
                if '1_' in file : ds1= pd.read_csv(r+'/'+file, sep =sep_for_source_files, engine='python')
                if '2_' in file:  ds2= pd.read_csv(r+'/'+file, sep =sep_for_source_files, engine='python')

            if not ds1.empty and not ds2.empty and not gs.empty: break    


        if not ds1.empty and not ds2.empty and not gs.empty:
            ds1['subject_id'] = ds1['subject_id'].apply(str)
            ds2['subject_id'] = ds2['subject_id'].apply(str)

            if (source_folder=="leipzig_datasets" or source_folder=="corleone_datasets"):
                ds1['subject_id'] = ds1['subject_id'].str.lower()
                ds2['subject_id'] = ds2['subject_id'].str.lower()


            gs['source_id'] = gs['source_id'].apply(str)
            gs['target_id'] = gs['target_id'].apply(str)

            if path.exists(r+"/feature_vector.csv"):
                feature_vector = pd.read_csv(r+"/feature_vector.csv", sep =",", engine='python')
            else: 
                feature_vector = createFeatureVectorFile(ds1,ds2,gs, embeddings = False, printProgress=True, 
                                                         saveFile=r+"/feature_vector.csv", threads=threads)


            common_attributes = [value for value in ds1.columns if (value in ds2.columns and value!='subject_id')] 

            matching_task = MatchingTask(ds1, ds2, gs, feature_vector, dict_.get(dataset_name), common_attributes, feature_vector_train,feature_vector_test,feature_vector_validation)
            if (summaryFeatures):
                matching_task.getSummaryFeatures()
                #correspondes features
                summary_features = matching_task.dict_summary
                summary_features['Dataset'] = dataset_ext_name
                summary_features['Domain'] = dataset_domain
                for key in matching_tasks_summary.columns:
                    matching_tasks_summary.loc[dat_counter, key] = summary_features.get(key)


            if (baselineResults):
                # get baseline results
                if (train_test_val): 
                    print("Evaluation with train_validation_test split")
                    matching_task.getSplitValidationResults(model="linear")
                    matching_task.getSplitValidationResults(model="non-linear")
                else:
                    print("Evaluation with Nested-X-Validation (no splits will be considered) - slow for large tasks")
                    matching_task.getNestedXValidationResults(model="linear")
                    matching_task.getNestedXValidationResults(model="non-linear")

                #linear model results
                linear_results = matching_task.dict_linear_results
                linear_results['Dataset'] = dataset_ext_name
                for key in linear_results:
                    matching_tasks_baseline_svm_results.loc[dat_counter, key] = linear_results.get(key)
                #non-linear model results
                non_linear_results = matching_task.dict_non_linear_results
                non_linear_results['Dataset'] = dataset_ext_name
                for key in non_linear_results:
                    matching_tasks_baseline_rf_results.loc[dat_counter, key] = non_linear_results.get(key)

            if(profilingFeatures):
                matching_task.getProfilingFeatures()
                ident_features_profile =  matching_task.dict_profiling_features
                ident_features_profile['Dataset'] = dataset_ext_name
                for key in matching_tasks_profiling.columns:
                    matching_tasks_profiling.loc[dat_counter,key] = str(ident_features_profile.get(key))                                    


            dat_counter+=1
    
    
def displaySaveResults(): 
    timestr = time.strftime("%m%d_%H%M%S")
    if (profilingFeatures):
        display(matching_tasks_summary)
        matching_tasks_summary.to_csv(main_path+source_folder+'/'+timestr+'matching_tasks_profiling.csv', index=False)

    if (baselineResults):
        display(matching_tasks_svm_results)
        display(matching_tasks_rf_results)
        matching_tasks_rf_results.to_csv(main_path+source_folder+'/'+timestr+'matching_tasks_RF_results.csv', index=False)
        matching_tasks_svm_results.to_csv(main_path+source_folder+'/'+timestr+'matching_tasks_SVM_results.csv', index=False)

    if(profilingFeatures):
        display(matching_tasks_profiling)
        matching_tasks_profiling.to_csv(main_path+source_folder+'/'+timestr+'matching_tasks_profiling_features_summary.csv', index=False)